In [1]:
#  Required Install
!pip install librosa

In [2]:
#Import Dependencies
from pylab import *
import os
import pandas as pd
import librosa
import librosa.display
import glob
import random

# Sample one audio File

In [3]:
data, sampling_rate = librosa.load('../data/emotion_speech/03-01-08-01-02-02-02.wav')
plt.figure(figsize=(12, 4))

print(data, sampling_rate)
librosa.display.waveplot(data, sr=sampling_rate)

[0. 0. 0. ... 0. 0. 0.] 22050


# RAVDASS AUDIO SET


# Generate list of file and apply emotion str

In [4]:
##Get list of Files in folder
file_name = []

for root, dirs, files in os.walk("../data/emotion_speech/."):
    for filename in files:
        file_name.append(filename)

file_name_df = pd.DataFrame(file_name, columns = ['file_name'])
file_name_df['encoding']= file_name_df['file_name']
file_name_df['ID'] = file_name_df.index 
file_name_df['dir']= 'emotion_speech'
file_name_df

,file_name,encoding,ID,dir
0,03-01-01-01-01-01-01.wav,03-01-01-01-01-01-01.wav,0,emotion_speech
1,03-01-01-01-01-01-02.wav,03-01-01-01-01-01-02.wav,1,emotion_speech
2,03-01-01-01-01-01-03.wav,03-01-01-01-01-01-03.wav,2,emotion_speech
3,03-01-01-01-01-01-04.wav,03-01-01-01-01-01-04.wav,3,emotion_speech
4,03-01-01-01-01-01-05.wav,03-01-01-01-01-01-05.wav,4,emotion_speech
...,...,...,...,...
1435,03-01-08-02-02-02-20.wav,03-01-08-02-02-02-20.wav,1435,emotion_speech
1436,03-01-08-02-02-02-21.wav,03-01-08-02-02-02-21.wav,1436,emotion_speech
1437,03-01-08-02-02-02-22.wav,03-01-08-02-02-02-22.wav,1437,emotion_speech
1438,03-01-08-02-02-02-23.wav,03-01-08-02-02-02-23.wav,1438,emotion_speech


In [39]:
#Create DF seperating Values
# Filename identifiers

# Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
# Vocal channel (01 = speech, 02 = song).
# Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
# Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
# Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
# Repetition (01 = 1st repetition, 02 = 2nd repetition).
# Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

encoding_df = file_name_df["encoding"].str.split("-", n=-1, expand=True)
encoding_df.columns = ['Modality','Vocal_channel','Class','Intensity','Statement','Repetition','Actor']

"""
1. Added a new column - Gender
2. Populated Gender column after stripping .wav from 'Actor'
3. Convert data type from obj to string then to int which is 
   needed to determine odd or even
"""
encoding_df['Actor'] = encoding_df['Actor'].map(lambda actor: actor.rstrip('.wav'))
encoding_df['Gender'] = 'Gender'
encoding_df['Actor'].astype(str).astype(int)

encoding_df
"""
1. Loop through each row in dataframe 'Actor' column
2. Set value of Gender to Female if Actor value is even or Male if odd
"""
for index, row in encoding_df.iterrows():
    if int(row[6]) % 2 == 0:
        row['Gender'] = '1'
    else:
        row['Gender'] = '0'
        
encoding_df
final_df = encoding_df.drop(columns=['Modality','Vocal_channel','Intensity','Statement','Repetition','Actor'])

final_df.loc[final_df.Class == '01', 'Class'] = '02'
final_df.loc[final_df.Class == '02', 'Emotion'] = 'calm'
final_df.loc[final_df.Class == '03', 'Emotion'] = 'happy'
final_df.loc[final_df.Class == '04', 'Emotion'] = 'sad'
final_df.loc[final_df.Class == '05', 'Emotion'] = 'angry'
final_df.loc[final_df.Class == '06', 'Emotion'] = 'fearful'
final_df.loc[final_df.Class == '07', 'Emotion'] = 'disgust'
final_df.loc[final_df.Class == '08', 'Emotion'] = 'surprise'

final_df

,Class,Gender,Emotion
0,02,0,calm
1,02,1,calm
2,02,0,calm
3,02,1,calm
4,02,0,calm
...,...,...,...
1435,08,1,surprise
1436,08,0,surprise
1437,08,1,surprise
1438,08,0,surprise


In [40]:
#merge data frame
pre_merged_df = file_name_df.join(final_df, how='outer')
merged_df = pre_merged_df.drop(columns=['encoding'])
merged_df.set_index('ID')

,file_name,dir,Class,Gender,Emotion
ID,,,,,
0,03-01-01-01-01-01-01.wav,emotion_speech,02,0,calm
1,03-01-01-01-01-01-02.wav,emotion_speech,02,1,calm
2,03-01-01-01-01-01-03.wav,emotion_speech,02,0,calm
3,03-01-01-01-01-01-04.wav,emotion_speech,02,1,calm
4,03-01-01-01-01-01-05.wav,emotion_speech,02,0,calm
...,...,...,...,...,...
1435,03-01-08-02-02-02-20.wav,emotion_speech,08,1,surprise
1436,03-01-08-02-02-02-21.wav,emotion_speech,08,0,surprise
1437,03-01-08-02-02-02-22.wav,emotion_speech,08,1,surprise


In [41]:
merged_df.Class.value_counts()

02    288
04    192
05    192
06    192
07    192
08    192
03    192
Name: Class, dtype: int64

# TORONTO EMOTIONAL SPEECH DATASET

In [42]:
##Get list of Files in folder
t_file_name = []

for root, dirs, files in os.walk("../data/toronto_speech/."):
    for filename in files:
        t_file_name.append(filename)

t_file_name_df = pd.DataFrame(t_file_name, columns = ['file_name'])
t_file_name_df['encoding']= t_file_name_df['file_name']
t_file_name_df['ID'] = t_file_name_df.index 
t_file_name_df['dir'] = 'toronto_speech'
t_file_name_df.head(20)

,file_name,encoding,ID,dir
0,OAF_back_angry.wav,OAF_back_angry.wav,0,toronto_speech
1,OAF_back_disgust.wav,OAF_back_disgust.wav,1,toronto_speech
2,OAF_back_fear.wav,OAF_back_fear.wav,2,toronto_speech
3,OAF_back_happy.wav,OAF_back_happy.wav,3,toronto_speech
4,OAF_back_neutral.wav,OAF_back_neutral.wav,4,toronto_speech
5,OAF_back_ps.wav,OAF_back_ps.wav,5,toronto_speech
6,OAF_back_sad.wav,OAF_back_sad.wav,6,toronto_speech
7,OAF_bar_angry.wav,OAF_bar_angry.wav,7,toronto_speech
8,OAF_bar_disgust.wav,OAF_bar_disgust.wav,8,toronto_speech
9,OAF_bar_fear.wav,OAF_bar_fear.wav,9,toronto_speech


In [43]:
t_encoding_df = t_file_name_df["encoding"].str.split("_", n=-1, expand=True)
t_encoding_df.columns = ['Actor','Word','Emotion']
t_encoding_df['Gender'] = '1'
#t_encoding_df
t_final_df = t_encoding_df.drop(columns=['Actor','Word'])
t_final_df['Emotion'] = t_final_df['Emotion'].map(lambda emotion: emotion.rstrip('.wav'))

#t_final_df = t_final_df['Emotion'].str.strip('.wav')
t_final_df
toronto_df = pd.DataFrame(t_final_df)
toronto_df

,Emotion,Gender
0,angry,1
1,disgust,1
2,fear,1
3,happy,1
4,neutral,1
...,...,...
2795,fear,1
2796,happy,1
2797,neutral,1
2798,ps,1


In [44]:
#Fix odd values in file and convert Nuetral class to calm
toronto_df.loc[toronto_df.Emotion == 'ps', 'Emotion'] = 'surprise'
toronto_df.loc[toronto_df.Emotion == 'alm', 'Emotion'] = 'calm'
toronto_df.loc[toronto_df.Emotion == 'ngry', 'Emotion'] = 'angry'
toronto_df.loc[toronto_df.Emotion == 'neutral', 'Emotion'] = 'calm'
toronto_df.loc[toronto_df.Emotion == 'fear', 'Emotion'] = 'fearful'

#assign emotion value numbers
toronto_df.loc[toronto_df.Emotion == 'calm', 'Class'] = '02'
toronto_df.loc[toronto_df.Emotion == 'happy', 'Class'] = '03'
toronto_df.loc[toronto_df.Emotion == 'sad', 'Class'] = '04'
toronto_df.loc[toronto_df.Emotion == 'angry', 'Class'] = '05'
toronto_df.loc[toronto_df.Emotion == 'fearful', 'Class'] = '06'
toronto_df.loc[toronto_df.Emotion == 'disgust', 'Class'] = '07'
toronto_df.loc[toronto_df.Emotion == 'surprise', 'Class'] = '08'

toronto_df.head(15)

,Emotion,Gender,Class
0,angry,1,05
1,disgust,1,07
2,fearful,1,06
3,happy,1,03
4,calm,1,02
5,surprise,1,08
6,sad,1,04
7,angry,1,05
8,disgust,1,07
9,fearful,1,06


In [86]:
#merge data frame and export CSV
t_pre_merged_df = t_file_name_df.join(toronto_df, how='outer')
t_merged_df = t_pre_merged_df.drop(columns=['encoding'])
t_merged_df = t_merged_df[['file_name', 'dir', 'Class','Gender','Emotion','ID']]
t_merged_df
t_merged_df.set_index('ID')

,file_name,dir,Class,Gender,Emotion
ID,,,,,
0,OAF_back_angry.wav,toronto_speech,05,1,angry
1,OAF_back_disgust.wav,toronto_speech,07,1,disgust
2,OAF_back_fear.wav,toronto_speech,06,1,fearful
3,OAF_back_happy.wav,toronto_speech,03,1,happy
4,OAF_back_neutral.wav,toronto_speech,02,1,calm
...,...,...,...,...,...
2795,YAF_youth_fear.wav,toronto_speech,06,1,fearful
2796,YAF_youth_happy.wav,toronto_speech,03,1,happy
2797,YAF_youth_neutral.wav,toronto_speech,02,1,calm


In [87]:
t_merged_df.Class.value_counts()

05    400
08    400
02    400
04    400
06    400
07    400
03    400
Name: Class, dtype: int64

# CONCAT DATASETS & RESET INDEX

In [88]:
dataset_df = pd.concat([merged_df, t_merged_df])
dataset2 = dataset_df.reset_index(drop=True)
dataset2 = dataset2.drop(columns=['Class'])
dataset2['ID']= dataset2.index
final_dataset_df = dataset2
final_dataset_df

,file_name,ID,dir,Gender,Emotion
0,03-01-01-01-01-01-01.wav,0,emotion_speech,0,calm
1,03-01-01-01-01-01-02.wav,1,emotion_speech,1,calm
2,03-01-01-01-01-01-03.wav,2,emotion_speech,0,calm
3,03-01-01-01-01-01-04.wav,3,emotion_speech,1,calm
4,03-01-01-01-01-01-05.wav,4,emotion_speech,0,calm
...,...,...,...,...,...
4235,YAF_youth_fear.wav,4235,toronto_speech,1,fearful
4236,YAF_youth_happy.wav,4236,toronto_speech,1,happy
4237,YAF_youth_neutral.wav,4237,toronto_speech,1,calm
4238,YAF_youth_ps.wav,4238,toronto_speech,1,surprise


In [89]:
final_dataset_df.Emotion.value_counts()

calm        688
sad         592
disgust     592
happy       592
surprise    592
fearful     592
angry       592
Name: Emotion, dtype: int64

In [90]:
final_dataset_df.Gender.value_counts()

1    3520
0     720
Name: Gender, dtype: int64

In [91]:
final_dataset_df.to_csv(r'../data/gender_feature_data.csv', index = True)

In [92]:
train = pd.read_csv('../data/gender_feature_data.csv')
data_dir = ('../data')
train.head()

,Unnamed: 0,file_name,ID,dir,Gender,Emotion
0,0,03-01-01-01-01-01-01.wav,0,emotion_speech,0,calm
1,1,03-01-01-01-01-01-02.wav,1,emotion_speech,1,calm
2,2,03-01-01-01-01-01-03.wav,2,emotion_speech,0,calm
3,3,03-01-01-01-01-01-04.wav,3,emotion_speech,1,calm
4,4,03-01-01-01-01-01-05.wav,4,emotion_speech,0,calm


# CREATE FEATURES

In [93]:
train.Gender.value_counts()

1    3520
0     720
Name: Gender, dtype: int64

In [94]:
def parser(row):
   # function to load files and extract features
   file_name = os.path.join(os.path.abspath(data_dir), str(row.dir), str(row.file_name))

   # handle exception to check if there isn't a file which is corrupted
   try:
      # extraction
      X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
      # extract mfcc data
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
   except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None, None
 
   feature = mfccs
   label = row.Gender
   emotion = row.Emotion

 
   return [feature, label, emotion]

temp = train.apply(parser, axis=1)

In [95]:
features = [x[0] for x in temp]
features

labels = [x[1] for x in temp]
labels

emotions = [x[2] for x in temp]
emotions

['calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 'calm',
 

In [96]:
features_df = pd.DataFrame(data=features)
features_df

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,-6.381716,-0.188635,-13.735003,-0.319724,...,-1.411359,-2.769772,-2.042009,-2.522663,-2.507448,-2.250499,-0.381507,-2.481059,-2.791023,-2.244865
1,-653.169006,58.028076,-12.581207,11.818786,-7.681562,-0.617142,-8.337758,-5.823570,-6.547592,1.458057,...,0.788761,2.570493,2.558634,3.708506,2.790020,2.201920,-1.021456,0.819201,-0.277811,0.207586
2,-617.052307,60.103371,-5.984258,13.886285,1.120427,0.511750,-14.841358,-4.016369,-5.575839,-6.309851,...,-1.133367,-2.019846,-1.024786,0.331096,0.531833,-1.621019,-2.158077,-2.502774,-0.676133,2.089097
3,-698.721130,47.088486,-11.333612,12.963088,-8.005651,-1.252229,-10.009068,-9.434125,-10.318875,-0.864674,...,-0.299306,-0.004270,-1.408215,0.958172,2.574661,1.877764,1.608618,0.222401,4.338524,4.266795
4,-715.922607,71.976799,4.457525,18.602499,8.404046,3.621954,-1.974365,0.022673,-2.678083,4.232803,...,0.505697,0.424780,0.972097,0.210004,0.667769,1.918956,0.174543,-1.768463,-1.512280,-0.253279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,-343.403870,32.597549,-9.197987,25.315771,0.052862,0.853997,-11.733653,0.140329,-25.528193,4.334095,...,3.723363,7.743777,2.505844,-6.495988,-7.261127,1.248809,3.646969,-0.981628,0.113497,5.193588
4236,-345.477142,24.427334,-14.398651,36.952991,-7.297602,1.008584,-15.700944,-10.525764,-6.183089,-6.078100,...,7.658078,-2.222577,-0.601163,4.032121,8.394070,5.845022,-0.454776,-2.956181,-1.036385,-1.652752
4237,-416.726318,71.149239,-1.116021,42.524796,-6.296183,2.258823,-6.628645,-7.701837,-10.190815,7.239936,...,-6.868152,7.055337,19.378656,29.953960,34.230598,25.548527,10.954952,-3.200483,-4.406600,0.721583
4238,-374.824036,46.681873,-12.381617,18.139082,-22.662424,4.170814,-15.546303,-7.580999,-18.976576,6.013675,...,3.868166,7.087386,5.132565,5.092512,5.456816,7.219532,5.145615,4.012959,3.082388,1.417208


In [97]:
features_df["label"] = labels
features_df["emotions"] = emotions

features_df

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,label,emotions
0,-700.398926,58.630211,-3.025852,16.040241,4.248529,3.869935,-6.381716,-0.188635,-13.735003,-0.319724,...,-2.042009,-2.522663,-2.507448,-2.250499,-0.381507,-2.481059,-2.791023,-2.244865,0,calm
1,-653.169006,58.028076,-12.581207,11.818786,-7.681562,-0.617142,-8.337758,-5.823570,-6.547592,1.458057,...,2.558634,3.708506,2.790020,2.201920,-1.021456,0.819201,-0.277811,0.207586,1,calm
2,-617.052307,60.103371,-5.984258,13.886285,1.120427,0.511750,-14.841358,-4.016369,-5.575839,-6.309851,...,-1.024786,0.331096,0.531833,-1.621019,-2.158077,-2.502774,-0.676133,2.089097,0,calm
3,-698.721130,47.088486,-11.333612,12.963088,-8.005651,-1.252229,-10.009068,-9.434125,-10.318875,-0.864674,...,-1.408215,0.958172,2.574661,1.877764,1.608618,0.222401,4.338524,4.266795,1,calm
4,-715.922607,71.976799,4.457525,18.602499,8.404046,3.621954,-1.974365,0.022673,-2.678083,4.232803,...,0.972097,0.210004,0.667769,1.918956,0.174543,-1.768463,-1.512280,-0.253279,0,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,-343.403870,32.597549,-9.197987,25.315771,0.052862,0.853997,-11.733653,0.140329,-25.528193,4.334095,...,2.505844,-6.495988,-7.261127,1.248809,3.646969,-0.981628,0.113497,5.193588,1,fearful
4236,-345.477142,24.427334,-14.398651,36.952991,-7.297602,1.008584,-15.700944,-10.525764,-6.183089,-6.078100,...,-0.601163,4.032121,8.394070,5.845022,-0.454776,-2.956181,-1.036385,-1.652752,1,happy
4237,-416.726318,71.149239,-1.116021,42.524796,-6.296183,2.258823,-6.628645,-7.701837,-10.190815,7.239936,...,19.378656,29.953960,34.230598,25.548527,10.954952,-3.200483,-4.406600,0.721583,1,calm
4238,-374.824036,46.681873,-12.381617,18.139082,-22.662424,4.170814,-15.546303,-7.580999,-18.976576,6.013675,...,5.132565,5.092512,5.456816,7.219532,5.145615,4.012959,3.082388,1.417208,1,surprise


In [98]:
#Write to CSV
features_df.to_csv('../data/gender_data.csv', index=False)